In [1]:
from gurobipy import *
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

In [2]:
# Create constants
dt = 0.1
o = np.array([[-1.569474], [-2.90308]])
og = np.array([[-0.162659], [-0.073026]])
B = np.array([[-0.028864, 0.150231], [-0.109971, 0.012843]])
C = np.array([[7.591472, -27.43076 ], [-22.424526, -6.139622]])
D = np.array([[3.07829, -0.330682], [-0.075344, 5.513974]])

In [19]:
m = Model("model")
m.setParam('OutputFlag', 0) 

u1 = m.addVar(name="u1", lb=-GRB.INFINITY, ub=GRB.INFINITY)
u2 = m.addVar(name="u2", lb=-GRB.INFINITY, ub=GRB.INFINITY)

u = np.array([[u1], [u2]])
distance = (og - (o + np.dot(B, o) + dt*np.dot(C, u)))
obj = np.dot(np.dot(distance.T, D), distance)[0, 0]
m.setObjective(obj, GRB.MINIMIZE)
M = 1
m.addQConstr(u1*u1 + u2*u2 <= M**2, "c0")

m.optimize()
[v.x for v in m.getVars()]

[-0.7363808424493474, -0.6765671688124775]

In [20]:
def obj_func(params):
    u = params.reshape(2,1)
    distance = (og - (o + np.dot(B, o) + dt*np.dot(C, u)))
    obj = np.dot(np.dot(distance.T, D), distance)
    return obj
    
max_speed_constraint =  {'type': 'ineq', 'fun': lambda u: M - np.linalg.norm(u)}
result = optimize.minimize(obj_func, x0=np.array([[0], [0]]), constraints=max_speed_constraint)
if result.success:
    print(result.x)
else:
    print(result)

[-0.73633273 -0.67661962]


# Which is faster? Gurobi or Scipy?

In [21]:
%%timeit -n10000
# Gurobi
m.optimize()

10000 loops, best of 3: 174 µs per loop


In [22]:
%%timeit -n10000
# Scipy
optimize.minimize(obj_func, x0=np.array([[0], [0]]), constraints=max_speed_constraint)

10000 loops, best of 3: 620 µs per loop
